In [1]:
import pandas as pd
import numpy as np
import time

today = time.strftime('%Y%m%d')

## Create p10_pba50_EIR_attr

In [2]:
p10_pba50_FBP_attr = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Policies\Zoning Modifications\p10_pba50_FBP_attr_20201110.csv')

print('p10_pba50_FBP_attr has {} rows, {} unique PARCEL_IDs'.format(p10_pba50_FBP_attr.shape[0],
                                                                    len(p10_pba50_FBP_attr.PARCEL_ID.unique())))
print('p10_pba50_FBP_attr fields: {}'.format(list(p10_pba50_FBP_attr)))

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


p10_pba50_FBP_attr has 1956208 rows, 1956208 unique PARCEL_IDs
p10_pba50_FBP_attr fields: ['OBJECTID', 'PARCEL_ID', 'DEVELOPMEN', 'LAND_VALUE', 'ACRES', 'COUNTY_ID', 'ZONE_ID', 'PROPORTION', 'TAX_EXEMPT', 'APN', 'GEOM_ID', 'IMPUTATION', 'ID', 'CENTROID', 'X', 'Y', 'geom_id_s', 'manual_cou', 'jurisdicti', 'pda_id', 'tpp_id', 'exp_id', 'exp_score', 'opp_id', 'zoningmodc', 'perffoot', 'perfarea', 'mapshell', 'tpa_id', 'perfarea2', 'alt_zoning', 'zonetype', 'pubopp_id', 'puboppuse', 'juris_id', 'hra_id', 'trich_id', 'cat_id', 'chcat', 'zoninghzca', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 'pba50zonin', 'nodev', 'juris', 'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_', 'fbp_ppa_id', 'fbp_exp202', 'fbpchcat', 'fbp_exsfd_', 'fbpchcatws', 'fbpzoningm', 'Shape_Leng', 'Shape_Area']


In [3]:
# create "p10_pba50_EIR_attr", starting with copying "p10_pba50_FBP_attr" 
p10_pba50_EIR_attr = p10_pba50_FBP_attr.copy()
p10_pba50_EIR_attr['PARCEL_ID'] = p10_pba50_EIR_attr['PARCEL_ID'].apply(lambda x: int(round(x)))

# create columns for EIR variables, starting with copying values from correponding FBP fields

fbp_cols = ['fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_',
            'fbp_ppa_id', 'fbp_exp202']

for colname_fbp in fbp_cols:
    colname_eir = colname_fbp.replace('fbp','eir')
    p10_pba50_EIR_attr[colname_eir] = p10_pba50_EIR_attr[colname_fbp]


# read p10-CoC tagging and merge into p10_pba50_EIR_attr
p10_cocid = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Policies\Zoning Modifications\p10_pba50_cocid.csv')

# merge "coc_id" field to "p10_pba50_EIR_attr"
p10_pba50_EIR_attr = p10_pba50_EIR_attr.merge(p10_cocid, on='PARCEL_ID', how='outer')
p10_pba50_EIR_attr.rename(columns={'coc_id':'eir_coc_id'}, inplace=True)
print('p10_pba50_EIR_attr has {} rows, {} unique PARCEL_IDs'.format(p10_pba50_EIR_attr.shape[0],
                                                                    len(p10_pba50_EIR_attr.PARCEL_ID.unique())))

# read existing_res_bldg tagging and merge into p10_pba50_EIR_attr
p10_ex_res = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Policies\Zoning Modifications\p10_existing_res_bldg.csv')
p10_pba50_EIR_attr = p10_pba50_EIR_attr.merge(p10_ex_res, on='PARCEL_ID', how='outer')
print('p10_pba50_EIR_attr has {} rows, {} unique PARCEL_IDs'.format(p10_pba50_EIR_attr.shape[0],
                                                                    len(p10_pba50_EIR_attr.PARCEL_ID.unique())))

p10_pba50_EIR_attr has 1956208 rows, 1956208 unique PARCEL_IDs
p10_pba50_EIR_attr has 1956208 rows, 1956208 unique PARCEL_IDs


In [4]:
# add "eirzoningm" field using the new concatenation

eir_zmodcat_cols = ['juris','eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 'eir_ppa_id', 'eir_exp202', 'ex_res_bldg']

p10_pba50_EIR_attr[eir_zmodcat_cols] = p10_pba50_EIR_attr[eir_zmodcat_cols].apply(lambda x: x.fillna('NA'))
p10_pba50_EIR_attr['eirzoningm'] = p10_pba50_EIR_attr[eir_zmodcat_cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
print('{} unique eirzoningmod:'.format(len(p10_pba50_EIR_attr.eirzoningm.unique())))
display(p10_pba50_EIR_attr.eirzoningm.unique())

print('p10_pba50_EIR_attr has the following fields: {}'.format(list(p10_pba50_EIR_attr)))

3535 unique eirzoningmod:


array(['livermoreNANAHRADISNANAinNA', 'livermoreNAtra3DISNANAinNA',
       'haywardNANANANANAinres', ..., 'napaNANANANANAexp1res',
       'unincorporated_napaNANANANAppainunNA',
       'unincorporated_marinNANADISNANAubz_npNA'], dtype=object)

p10_pba50_EIR_attr has the following fields: ['OBJECTID', 'PARCEL_ID', 'DEVELOPMEN', 'LAND_VALUE', 'ACRES', 'COUNTY_ID', 'ZONE_ID', 'PROPORTION', 'TAX_EXEMPT', 'APN', 'GEOM_ID', 'IMPUTATION', 'ID', 'CENTROID', 'X', 'Y', 'geom_id_s', 'manual_cou', 'jurisdicti', 'pda_id', 'tpp_id', 'exp_id', 'exp_score', 'opp_id', 'zoningmodc', 'perffoot', 'perfarea', 'mapshell', 'tpa_id', 'perfarea2', 'alt_zoning', 'zonetype', 'pubopp_id', 'puboppuse', 'juris_id', 'hra_id', 'trich_id', 'cat_id', 'chcat', 'zoninghzca', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 'pba50zonin', 'nodev', 'juris', 'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_', 'fbp_ppa_id', 'fbp_exp202', 'fbpchcat', 'fbp_exsfd_', 'fbpchcatws', 'fbpzoningm', 'Shape_Leng', 'Shape_Area', 'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_ppa_id', 'eir_exp202', 'eir_coc_id', 'ex_res_bldg', 'eirzoningm']


In [5]:
# export
# p10_pba50_EIR_attr.to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_pba50_EIR_attr_{}.csv'.format(today), index=False)

## create zoning_mods lookup table

In [6]:
# bring in zoningmods fields from FBP as place holders

# read these fields in s24
lookup_fbp = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\bayarea_urbansim\data\zoning_mods_24.csv',
                         usecols = ['fbpzoningmodcat', 'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                    'dua_down', 'far_down', 'subsidy', 'notes', 'res_rent_cat', 'job_out_cat'])

print('zoning_mods_24 has {} unique fbpzoningmodcat'.format(lookup_fbp.shape[0]))
display(lookup_fbp.head())
#print(list(lookup_fbp))
print('dua_up has the following values: {}'.format(list(lookup_fbp.dua_up.unique())))
print('dua_down has the following values: {}'.format(list(lookup_fbp.dua_down.unique())))
print('far_up has the following values: {}'.format(list(lookup_fbp.far_up.unique())))
print('far_down has the following values: {}'.format(list(lookup_fbp.far_down.unique())))
print('add_bldg has the following values: {}'.format(list(lookup_fbp.add_bldg.unique())))
print('drop_bldg has the following values: {}'.format(list(lookup_fbp.drop_bldg.unique())))

p10_pba50_EIR_attr_modcat = p10_pba50_EIR_attr.merge(lookup_fbp, 
                                                     left_on='fbpzoningm',
                                                     right_on='fbpzoningmodcat', how='left')
print('p10_pba50_EIR_attr_modcat has {} rows'.format(p10_pba50_EIR_attr_modcat.shape[0]))

zoning_mods_24 has 2970 unique fbpzoningmodcat


,fbpzoningmodcat,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,subsidy,notes,res_rent_cat,job_out_cat
0,alamedaGGNADISNAinNA,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alamedaGGNADISNAinsfd,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alamedaGGNAHRADISNAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alamedaGGNAHRADISNAinsfd,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alamedaGGNAHRANAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


dua_up has the following values: [25.0, 35.0, 75.0, 50.0, nan, 100.0, 200.0, 150.0, 2.75, 1.5]
dua_down has the following values: [nan, 0.0]
far_up has the following values: [nan, 2.0, 9.0, 3.0]
far_down has the following values: [nan, 0.0]
add_bldg has the following values: [nan, 'HM', 'IW', 'HS']
drop_bldg has the following values: [nan, 'HM']
p10_pba50_EIR_attr_modcat has 1956208 rows


In [7]:
# collapsed to lookup table based on 'eirzoningm' and EIR geography fields, 'fbpzoningmodcat'
#      was kept to inherent Final Blueprint values 

EIR_modcat_df = p10_pba50_EIR_attr_modcat[['ACRES', 'fbpzoningmodcat', 'eirzoningm', 'juris',
                                           'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                                           'eir_ppa_id', 'eir_exp202', 'ex_res_bldg',
                                           'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                           'dua_down', 'far_down', 'subsidy', 'res_rent_cat', 'job_out_cat']]

EIR_modcat_df = EIR_modcat_df[['eirzoningm', 'juris',
                               'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                               'eir_ppa_id', 'eir_exp202', 'ex_res_bldg',
                               'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                               'dua_down', 'far_down', 'subsidy','res_rent_cat', 'job_out_cat']].drop_duplicates()

# rename columns
EIR_modcat_df.rename(columns = {'eir_gg_id': 'gg_id',
                                'eir_tra_id': 'tra_id',
                                'eir_sesit_': 'sesit_id',
                                'eir_coc_id': 'coc_id',
                                'eir_ppa_id': 'ppa_id',
                                'eir_exp202': 'exp2020_id'}, inplace=True)

# add 'manual_county' column
juris_county = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\petrale\zones\jurisdictions\juris_county_id.csv',
                           usecols = ['juris_name_full', 'county_id'])
juris_county.columns = ['juris','manual_county']

EIR_modcat_df = EIR_modcat_df.merge(juris_county, on='juris', how='left')

In [8]:
## Update "dua_up" and "add_bldg" for residential - EIR Alt1 H3 strategy

EIR_modcat_alt1 = EIR_modcat_df.copy()

# first, set to nan
EIR_modcat_alt1.dua_up = np.nan
EIR_modcat_alt1.loc[(EIR_modcat_alt1.add_bldg == 'HM') | (EIR_modcat_alt1.add_bldg == 'HS'), 'add_bldg'] = np.nan

# update values
EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra1'), 'dua_up'] = 300
EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra1'), 'add_bldg'] = 'HM'

EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra2a'), 'dua_up'] = 250
EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra2a'), 'add_bldg'] = 'HM'

EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra2b'), 'dua_up'] = 200
EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra2b'), 'add_bldg'] = 'HM'

EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra2c'), 'dua_up'] = 150
EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra2c'), 'add_bldg'] = 'HM'

EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra3'), 'dua_up'] = 100
EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra3'), 'add_bldg'] = 'HM'

# Dave suggested not upzone non-TRA areas
# EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='NA'), 'dua_up'] = 50
# EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='NA'), 'add_bldg'] = 'HM'

In [9]:
## Update "dua_down" for residential - EIR Alt1 UGB strategy

no_dua_down_fbp = EIR_modcat_alt1.loc[EIR_modcat_alt1.dua_down.isnull()]
display(no_dua_down_fbp.exp2020_id.value_counts())

# zoningmods with 'inun' is the 'exp2020_id' field should have 'dua_down = 0'
EIR_modcat_alt1.loc[EIR_modcat_alt1.exp2020_id == 'inun', 'dua_down'] = 0

# check - there should only be 'in'
display(EIR_modcat_alt1.loc[EIR_modcat_alt1.dua_down.isnull()].exp2020_id.value_counts())

in      2526
inun     393
Name: exp2020_id, dtype: int64

in    2526
Name: exp2020_id, dtype: int64

In [10]:
# Update "far_up" and "add_bldg" for non-residential - EIR Alt1 H3 strategy

# first, set to nan
EIR_modcat_alt1.far_up = np.nan
EIR_modcat_alt1.loc[EIR_modcat_alt1.add_bldg == 'IW', 'add_bldg'] = np.nan

# update values
EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra1'), 'far_up'] = 9

EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (
                     EIR_modcat_alt1.ppa_id=='ppa') & (
                     EIR_modcat_alt1.tra_id != 'tra1'), 'far_up'] = 2

EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (
                     EIR_modcat_alt1.ppa_id=='ppa') & (
                     EIR_modcat_alt1.tra_id != 'tra1'), 'add_bldg'] = 'IW'

In [11]:
# Update "far_down" for non-residential - EIR Alt1 UGB strategy

no_far_down_fbp = EIR_modcat_alt1.loc[EIR_modcat_alt1.far_down.isnull()]
display(no_far_down_fbp.exp2020_id.value_counts())

# zoningmods with 'inun' is the 'exp2020_id' field should have 'dua_down = 0'
EIR_modcat_alt1.loc[EIR_modcat_alt1.exp2020_id == 'inun', 'far_down'] = 0

# check - there should only be 'in'
display(EIR_modcat_alt1.loc[EIR_modcat_alt1.far_down.isnull()].exp2020_id.value_counts())

in      2526
inun     393
Name: exp2020_id, dtype: int64

in    2526
Name: exp2020_id, dtype: int64

In [12]:
# drop duplicates
EIR_modcat_alt1 = EIR_modcat_alt1.drop_duplicates()
print('EIR_modcat_alt1 has {} rows'.format(EIR_modcat_alt1.shape[0]))

# add 'FREQUENCE', 'SUM_ACRES' columns
EIR_modcat_stats = p10_pba50_EIR_attr_modcat.groupby('eirzoningm').agg({'ACRES': ['count','sum']}).reset_index()
EIR_modcat_stats.columns = ['eirzoningm', 'FREQUENCY', 'SUM_ACRES']
print('EIR_modcat_stats has {} rows'.format(EIR_modcat_stats.shape[0]))

EIR_modcat_alt1 = EIR_modcat_alt1.merge(EIR_modcat_stats, on='eirzoningm', how='left')
print('p10_pba50_EIR_modcat_df has {} rows'.format(EIR_modcat_alt1.shape[0]))

# add 'modcat_id' column
EIR_modcat_alt1['modcat_id'] = EIR_modcat_alt1.index + 1

# recoder the fields
EIR_modcat_alt1 = EIR_modcat_alt1[['eirzoningm', 'modcat_id', 'FREQUENCY', 'SUM_ACRES', 'manual_county', 'juris', 
                                   'gg_id', 'tra_id', 'sesit_id', 'coc_id', 'ppa_id', 'exp2020_id', 'ex_res_bldg', 
                                   'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 'dua_down', 'far_down', 'subsidy', 'res_rent_cat', 'job_out_cat']]

EIR_modcat_alt1 has 3535 rows
EIR_modcat_stats has 3535 rows
p10_pba50_EIR_modcat_df has 3535 rows


In [13]:
# export
EIR_modcat_alt1.rename(columns={'eirzoningm': 'eirzoningmodcat'}, inplace=True)
print('export zoning_mods lookup table of {} rows'.format(EIR_modcat_alt1.shape[0]))
EIR_modcat_alt1.to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\BAUS input files\\zoning_mods_26_{}.csv'.format(today), index=False)

export zoning_mods lookup table of 3535 rows


In [14]:
EIR_modcat_alt1

,eirzoningmodcat,modcat_id,FREQUENCY,SUM_ACRES,manual_county,juris,gg_id,tra_id,sesit_id,coc_id,...,ex_res_bldg,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,subsidy,res_rent_cat,job_out_cat
0,livermoreNANAHRADISNANAinNA,1,128,394.404646,1,livermore,NA,NA,HRADIS,NA,...,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,livermoreNAtra3DISNANAinNA,2,1309,1676.670219,1,livermore,NA,tra3,DIS,NA,...,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,haywardNANANANANAinres,3,3607,680.363009,1,hayward,NA,NA,NA,NA,...,res,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,unincorporated_sonomaNANAHRADISNANAoutNA,4,2004,137054.547676,97,unincorporated_sonoma,NA,NA,HRADIS,NA,...,NA,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
4,fremontNANAHRADISNANAinNA,5,3425,2109.114080,1,fremont,NA,NA,HRADIS,NA,...,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,cloverdaleNANANANANAoutNA,3531,1,1.071807,97,cloverdale,NA,NA,NA,NA,...,NA,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3531,campbellNAtra2cHRANANAinNA,3532,1,2.044626,85,campbell,NA,tra2c,HRA,NA,...,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3532,napaNANANANANAexp1res,3533,1,20.309329,55,napa,NA,NA,NA,NA,...,res,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
3533,unincorporated_napaNANANANAppainunNA,3534,1,6.685013,55,unincorporated_napa,NA,NA,NA,NA,...,NA,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


## create zoning_mods lookup table for Alt2 (repeat steps above)

In [2]:
#read p10_pba50_EIR_attr
p10_pba50_EIR_attr = pd.read_csv('C:\\Users\\blu\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_pba50_EIR_attr_20210224.csv')

C:\Users\blu\Apps\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (23,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# bring in zoningmods fields from FBP as place holders

# read these fields in s24
lookup_fbp = pd.read_csv(r'C:\Users\blu\Documents\GitHub\bayarea_urbansim\data\zoning_mods_24.csv',
                         usecols = ['fbpzoningmodcat', 'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                    'dua_down', 'far_down', 'subsidy', 'notes', 'res_rent_cat', 'job_out_cat'])

print('zoning_mods_24 has {} unique fbpzoningmodcat'.format(lookup_fbp.shape[0]))
display(lookup_fbp.head())
#print(list(lookup_fbp))
print('dua_up has the following values: {}'.format(list(lookup_fbp.dua_up.unique())))
print('dua_down has the following values: {}'.format(list(lookup_fbp.dua_down.unique())))
print('far_up has the following values: {}'.format(list(lookup_fbp.far_up.unique())))
print('far_down has the following values: {}'.format(list(lookup_fbp.far_down.unique())))
print('add_bldg has the following values: {}'.format(list(lookup_fbp.add_bldg.unique())))
print('drop_bldg has the following values: {}'.format(list(lookup_fbp.drop_bldg.unique())))

p10_pba50_EIR_attr_modcat = p10_pba50_EIR_attr.merge(lookup_fbp, 
                                                     left_on='fbpzoningm',
                                                     right_on='fbpzoningmodcat', how='left')
print('p10_pba50_EIR_attr_modcat has {} rows'.format(p10_pba50_EIR_attr_modcat.shape[0]))

zoning_mods_24 has 2970 unique fbpzoningmodcat


,fbpzoningmodcat,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,subsidy,notes,res_rent_cat,job_out_cat
0,alamedaGGNADISNAinNA,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alamedaGGNADISNAinsfd,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alamedaGGNAHRADISNAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alamedaGGNAHRADISNAinsfd,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alamedaGGNAHRANAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


dua_up has the following values: [25.0, 35.0, 75.0, 50.0, nan, 100.0, 200.0, 150.0, 2.75, 1.5]
dua_down has the following values: [nan, 0.0]
far_up has the following values: [nan, 2.0, 9.0, 3.0]
far_down has the following values: [nan, 0.0]
add_bldg has the following values: [nan, 'HM', 'IW', 'HS']
drop_bldg has the following values: [nan, 'HM']
p10_pba50_EIR_attr_modcat has 1956208 rows


In [5]:
# collapsed to lookup table based on 'eirzoningm' and EIR geography fields, 'fbpzoningmodcat'
# was kept to inherent Final Blueprint values 

EIR_modcat_df = p10_pba50_EIR_attr_modcat[['ACRES', 'fbpzoningmodcat', 'eirzoningm', 'juris',
                                           'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                                           'eir_ppa_id', 'eir_exp202', 'ex_res_bldg',
                                           'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                           'dua_down', 'far_down', 'subsidy', 'res_rent_cat', 'job_out_cat']]

EIR_modcat_df = EIR_modcat_df[['eirzoningm', 'juris',
                               'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                               'eir_ppa_id', 'eir_exp202', 'ex_res_bldg',
                               'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                               'dua_down', 'far_down', 'subsidy','res_rent_cat', 'job_out_cat']].drop_duplicates()

# rename columns
EIR_modcat_df.rename(columns = {'eir_gg_id': 'gg_id',
                                'eir_tra_id': 'tra_id',
                                'eir_sesit_': 'sesit_id',
                                'eir_coc_id': 'coc_id',
                                'eir_ppa_id': 'ppa_id',
                                'eir_exp202': 'exp2020_id'}, inplace=True)

# add 'manual_county' column
juris_county = pd.read_csv(r'C:\Users\blu\Documents\GitHub\petrale\zones\jurisdictions\juris_county_id.csv',
                           usecols = ['juris_name_full', 'county_id'])
juris_county.columns = ['juris','manual_county']

EIR_modcat_df = EIR_modcat_df.merge(juris_county, on='juris', how='left')

EIR_modcat_df .head()

,OBJECTID,PARCEL_ID,DEVELOPMEN,LAND_VALUE,ACRES,COUNTY_ID,ZONE_ID,PROPORTION,TAX_EXEMPT,APN,...,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,subsidy,notes,res_rent_cat,job_out_cat
0,1,229116,1.0,0.0,3.360520,1.0,725.0,0.0,0.0,099 029001700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,244166,1.0,0.0,1.294423,1.0,715.0,0.0,0.0,099B540210200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,202378,25.0,6036500.0,14.993605,1.0,820.0,0.0,1.0,085A643106000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2004420,-1.0,179954.0,316.247146,97.0,1401.0,0.0,0.0,141-100-012,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,5,340332,2.0,0.0,0.621275,1.0,763.0,0.0,0.0,525 166004800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
EIR_modcat_df.add_bldg.unique()

array([nan, 'HM', 'HS', 'IW'], dtype=object)

In [12]:
EIR_modcat_df.columns

Index([u'eirzoningm', u'juris', u'gg_id', u'tra_id', u'sesit_id', u'coc_id',
       u'ppa_id', u'exp2020_id', u'ex_res_bldg', u'add_bldg', u'drop_bldg',
       u'dua_up', u'far_up', u'dua_down', u'far_down', u'subsidy',
       u'res_rent_cat', u'job_out_cat', u'manual_county'],
      dtype='object')

In [49]:
EIR_modcat_df.exp2020_id.unique()

array(['in', 'out', 'inun', 'exp1_np', 'expmax_np', 'exp3_au', 'exp1',
       'expmax', 'exp0', 'ubz', 'exp2_np', 'exp0_np', 'ubz_np'], dtype=object)

### Major changes in EIR Alt2 H3 strategy:
#### 1. allow residential development in GGs for res and non_res parcels
#### 2. change upzoning levels for different jurisdiction categories
#### 3. don't allow upzoning for res parcels in CoCs

In [16]:
#list of job Job-Rich & High-Resource Juris and Adjacent Juris
jlist = ['menlo_park', 'palo_alto', 'cupertino', 'milpitas',
         'atherton', 'belmont', 'campbell', 'east_palo_alto',
         'fremont', 'hayward', 'los_altos', 'los_altos_hills', 
         'los_gatos', 'monte_sereno', 'mountain_view', 'newark', 
         'redwood_city', 'portola_valley', 'san_carlos', 'san_jose', 
         'santa_clara', 'saratoga', 'sunnyvale', 'union_city', 'woodside']

In [54]:
EIR_modcat_alt2 = EIR_modcat_df.copy()

# first, set to nan
EIR_modcat_alt2.dua_up = np.nan
EIR_modcat_alt2.far_up = np.nan
EIR_modcat_alt2.add_bldg = np.nan

# create an HRA list
hra_list = ['HRA','HRADIS']

# update values for Residential zoning change

#tra1
## no difference among juris categories in tra1
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id=='tra1'), 'add_bldg'] = 'HM'
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id=='tra1'), 'dua_up'] = 125

#tra2
## first add HM to all parcels in tra2
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.str.contains('tra2', na = False)), 'add_bldg'] = 'HM'
## HRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.str.contains('tra2', na = False)) & (
                     EIR_modcat_alt2.sesit_id.isin(hra_list)), 'dua_up'] = 75

## adjust for juris in the list
### note: the following code doesn't differentiate HRA or nonHRA, because in the next step
### nonHRA gets lower upzoning, so the nonHRA in this juris list would get
### revised from 100 to 55 too
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.str.contains('tra2', na = False)) & (
                    EIR_modcat_alt2.juris.isin(jlist)), 'dua_up'] = 100

## nonHRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.str.contains('tra2', na = False)) & (
                    (EIR_modcat_alt2.sesit_id=='DIS') | (
                     EIR_modcat_alt2.sesit_id.isnull())), 'dua_up'] = 55

#tra3
## first add HM to all parcels in tra2
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id=='tra3'), 'add_bldg'] = 'HM'
## HRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id == 'tra3') & (
                     EIR_modcat_alt2.sesit_id.isin(hra_list)), 'dua_up'] = 50

## adjust for juris in the list
### note: the following code doesn't differentiate HRA or nonHRA, because in the next step
### nonHRA gets lower upzoning, so the nonHRA in this juris list would get
### revised from 75 to 50 too
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id == 'tra3') & (
                     EIR_modcat_alt2.juris.isin(jlist)), 'dua_up'] = 75

## nonHRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id == 'tra3') & (
                    (EIR_modcat_alt2.sesit_id=='DIS') | (
                     EIR_modcat_alt2.sesit_id.isnull())), 'dua_up'] = 35

# non-TRA
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.isnull()) & (
                     EIR_modcat_alt2.sesit_id.isin(hra_list)), 'add_bldg'] = 'HM'
## HRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.isnull()) & (
                     EIR_modcat_alt2.sesit_id.isin(hra_list)), 'dua_up'] = 50

## adjust for juris in the list
### note: the following code doesn't differentiate HRA or nonHRA, because in the next step
### nonHRA gets lower upzoning, so the nonHRA in this juris list would get
### revised from 75 to 50 too
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.isnull()) & (
                     EIR_modcat_alt2.juris.isin(jlist)), 'dua_up'] = 75

## nonHRA upzoning
EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.tra_id.isnull()) & (
                    (EIR_modcat_alt2.sesit_id=='DIS') | (
                     EIR_modcat_alt2.sesit_id.isnull())), 'dua_up'] = 35

In [56]:
#Unincorporated w/in UGB upzoning
EIR_modcat_alt2.loc[EIR_modcat_alt2.exp2020_id == 'inun', 'dua_up'] = 2
EIR_modcat_alt2.loc[EIR_modcat_alt2.exp2020_id == 'inun', 'add_bldg'] = 'HS'

In [59]:
# Maintain PPA zoning changes in FBP
#for now remove this one below to make sure housing gets built in core area first given the lower upzoning
#EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (EIR_modcat_alt2.tra_id=='tra1'), 'far_up'] = 9 

EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.ppa_id=='ppa') & (
                     EIR_modcat_alt2.tra_id != 'tra1'), 'far_up'] = 2

EIR_modcat_alt2.loc[(EIR_modcat_alt2.gg_id=='GG') & (
                     EIR_modcat_alt2.ppa_id=='ppa') & (
                     EIR_modcat_alt2.tra_id != 'tra1'), 'add_bldg'] = 'IW'

In [58]:
# limit development in CoCs
EIR_modcat_alt2.loc[(EIR_modcat_alt2.coc_id=='CoC') & (
                     EIR_modcat_alt2.ex_res_bldg =='res'), 'dua_up'] = np.nan
EIR_modcat_alt2.loc[(EIR_modcat_alt2.coc_id=='CoC') & (
                     EIR_modcat_alt2.ex_res_bldg =='res'), 'add_bldg'] = np.nan

In [60]:
# drop duplicates
EIR_modcat_alt2 = EIR_modcat_alt2.drop_duplicates()
print('EIR_modcat_alt2 has {} rows'.format(EIR_modcat_alt2.shape[0]))

# add 'FREQUENCE', 'SUM_ACRES' columns
EIR_modcat_stats = p10_pba50_EIR_attr_modcat.groupby('eirzoningm').agg({'ACRES': ['count','sum']}).reset_index()
EIR_modcat_stats.columns = ['eirzoningm', 'FREQUENCY', 'SUM_ACRES']
print('EIR_modcat_stats has {} rows'.format(EIR_modcat_stats.shape[0]))

EIR_modcat_alt2 = EIR_modcat_alt2.merge(EIR_modcat_stats, on='eirzoningm', how='left')
print('p10_pba50_EIR_modcat_df has {} rows'.format(EIR_modcat_alt2.shape[0]))

# add 'modcat_id' column
EIR_modcat_alt2['modcat_id'] = EIR_modcat_alt2.index + 1

# recoder the fields
EIR_modcat_alt2 = EIR_modcat_alt2[['eirzoningm', 'modcat_id', 'FREQUENCY', 'SUM_ACRES', 'manual_county', 'juris', 
                                   'gg_id', 'tra_id', 'sesit_id', 'coc_id', 'ppa_id', 'exp2020_id', 'ex_res_bldg', 
                                   'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 'dua_down', 'far_down', 'subsidy', 'res_rent_cat', 'job_out_cat']]

EIR_modcat_alt2 has 3535 rows
EIR_modcat_stats has 3535 rows
p10_pba50_EIR_modcat_df has 3535 rows


In [62]:
# export
EIR_modcat_alt2.rename(columns={'eirzoningm': 'eirzoningmodcat'}, inplace=True)
print('export zoning_mods lookup table of {} rows'.format(EIR_modcat_alt2.shape[0]))
EIR_modcat_alt2.to_csv('C:\\Users\\blu\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\BAUS input files\\zoning_mods_28_{}.csv'.format(today), index=False)

export zoning_mods lookup table of 3535 rows
